In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프 기본 테마 설정
# https://coldbrown.co.kr/2023/07/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%A4%EC%A0%84%ED%8E%B8-08-seaborn-sns-set%EC%9D%84-%ED%86%B5%ED%95%B4-%EC%8A%A4%ED%83%80%EC%9D%BC-%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0/
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False


# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats

In [7]:
status = pd.read_csv('../data/churn_status_전처리(민재님).csv')
status

,고객ID,분기,고객만족도점수,현재고객상태,이탈여부,이탈위험점수,고객생애가치,이탈유형,이탈사유
0,8779-QRDMV,Q3,3,Churned,True,91,5433,Competitor,Competitor offered more data
1,7495-OOKFY,Q3,3,Churned,True,69,5302,Competitor,Competitor made better offer
2,1658-BYGOY,Q3,2,Churned,True,81,3179,Competitor,Competitor made better offer
3,4598-XLKNJ,Q3,2,Churned,True,88,5337,Dissatisfaction,Limited range of services
4,4846-WHAFZ,Q3,2,Churned,True,67,2793,Price,Extra data charges
...,...,...,...,...,...,...,...,...,...
7038,2569-WGERO,Q3,5,Stayed,False,45,5306,NaN,NaN
7039,6840-RESVB,Q3,3,Stayed,False,59,2140,NaN,NaN
7040,2234-XADUH,Q3,4,Stayed,False,71,5560,NaN,NaN
7041,4801-JZAZL,Q3,4,Stayed,False,59,2793,NaN,NaN


In [11]:
total = pd.read_csv('../data/total_real_final.csv')
total

,고객ID,위도,경도,성별,고령자여부,배우자여부,부양가족여부,가입개월수,전화서비스가입여부,복수회선여부,...,프리미엄기술지원여부,음악스트리밍이용여부,무제한데이터이용여부,총환불액,총초과데이터요금,총장거리통화요금,총납부금,이탈유형,한글이탈유형,한글이탈이유
0,0002-ORFBO,34.827662,-118.999073,False,False,True,False,9,True,False,...,True,False,True,0.00,0,381.51,974.81,NaN,NaN,NaN
1,0003-MKNFE,34.162515,-118.203869,True,False,False,False,9,True,True,...,False,True,False,38.33,10,96.21,610.28,NaN,NaN,NaN
2,0004-TLHLJ,33.645672,-117.922613,True,False,False,False,4,True,False,...,False,False,True,0.00,0,134.60,415.45,Competitor,경쟁사,가격이 너무 높음
3,0011-IGKFF,38.014457,-122.115432,True,True,True,False,13,True,False,...,False,False,True,0.00,0,361.66,1599.51,Dissatisfaction,불만족,제품 불만족
4,0013-EXCHZ,34.227846,-119.079903,False,True,True,False,3,True,False,...,True,False,True,0.00,0,22.14,289.54,Dissatisfaction,불만족,네트워크 안정성
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,9987-LUTYD,32.759327,-116.997260,False,False,False,False,13,True,False,...,True,True,True,0.00,0,606.84,1349.74,NaN,NaN,NaN
7039,9992-RRAMN,37.734971,-120.954271,True,False,True,False,22,True,True,...,False,True,True,0.00,0,356.40,2230.10,Dissatisfaction,불만족,제품 불만족
7040,9992-UJOEL,39.108252,-123.645121,True,False,False,False,2,True,False,...,False,False,True,0.00,0,37.24,129.99,NaN,NaN,NaN
7041,9993-LHIEB,33.001813,-117.263628,True,False,True,False,67,True,False,...,True,True,True,0.00,0,142.04,4769.69,NaN,NaN,NaN


In [13]:
## 데이터프레임을 넣고 column별 특성 및 결측값, 고유값들을 확인하는 함수를 작성해본다.
## 필수는 아니지만 전체적인 흐름을 파악하기 쉬워진다.

def resumetable(df):
  print(f'데이터셋 크기: {df.shape}')                                # 데이터프레임의 전체 크기(행, 열) 출력

  summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])         # 각 피처의 데이터 타입을 가져와 데이터프레임으로 생성
  summary = summary.reset_index()                                    # 인덱스를 초기화하여 컬럼으로 변환
  summary = summary.rename(columns={'index':'피처'})                 # 'index' 컬럼명을 '피처'로 변경

  summary['결측값 개수'] = df.isnull().sum().values                 # 각 피처의 결측값(null) 개수 계산
  summary['고유값 개수'] = df.nunique().values                      # 각 피처의 고유값 개수 계산

  summary['첫 번째 값'] = df.loc[0].values                          # 각 피처의 첫 번째 샘플 값
  summary['두 번째 값'] = df.loc[1].values                          # 각 피처의 두 번째 샘플 값
  summary['세 번째 값'] = df.loc[2].values                          # 각 피처의 세 번째 샘플 값

  return summary                                                     # 요약 테이블 반환

In [15]:
resumetable(total)

데이터셋 크기: (7043, 46)


,피처,데이터 타입,결측값 개수,고유값 개수,첫 번째 값,두 번째 값,세 번째 값
0,고객ID,object,0,7043,0002-ORFBO,0003-MKNFE,0004-TLHLJ
1,위도,float64,0,1652,34.827662,34.162515,33.645672
2,경도,float64,0,1651,-118.999073,-118.203869,-117.922613
3,성별,bool,0,2,False,True,True
4,고령자여부,bool,0,2,False,False,False
5,배우자여부,bool,0,2,True,False,False
6,부양가족여부,bool,0,2,False,False,False
7,가입개월수,int64,0,73,9,9,4
8,전화서비스가입여부,bool,0,2,True,True,True
9,복수회선여부,bool,0,2,False,True,False


In [17]:
status_selected = status[['고객ID', '이탈유형']]
total_final = pd.merge(total, status_selected, on='고객ID', how = 'outer')
resumetable(total_final)

데이터셋 크기: (7043, 47)


,피처,데이터 타입,결측값 개수,고유값 개수,첫 번째 값,두 번째 값,세 번째 값
0,고객ID,object,0,7043,0002-ORFBO,0003-MKNFE,0004-TLHLJ
1,위도,float64,0,1652,34.827662,34.162515,33.645672
2,경도,float64,0,1651,-118.999073,-118.203869,-117.922613
3,성별,bool,0,2,False,True,True
4,고령자여부,bool,0,2,False,False,False
5,배우자여부,bool,0,2,True,False,False
6,부양가족여부,bool,0,2,False,False,False
7,가입개월수,int64,0,73,9,9,4
8,전화서비스가입여부,bool,0,2,True,True,True
9,복수회선여부,bool,0,2,False,True,False


In [23]:
total_real_final['이탈유형'].value_counts()

NameError: name 'total_real_final' is not defined

In [21]:
churntype_translation = {
    'Competitor': '경쟁사로 이탈',
    'Attitude': '태도 문제',
    'Dissatisfaction': '불만족',
    'Price': '가격 문제',
    'Other': '기타'
}

# 매핑 적용
total_final['한글이탈유형'] = total_final['이탈유형'].map(churntype_translation)

# 결과 확인
print(total_final[['이탈유형', '한글이탈유형']].head())

KeyError: '이탈유형'

In [ ]:
reason_mapping = {
    'Competitor made better offer': '경쟁사가 더 나은 제안을 했습니다',
    'Moved': '이동했습니다',
    'Competitor had better devices': '경쟁사가 더 나은 기기를 보유했습니다',
    'Competitor offered higher download speeds': '경쟁사가 더 빠른 다운로드 속도를 제공함',
    'Competitor offered more data': '경쟁사가 더 많은 데이터를 제공함',
    'Price too high': '가격이 너무 높음',
    'Product dissatisfaction': '제품 불만족',
    'Service dissatisfaction': '서비스 불만족',
    'Lack of self-service on Website': '웹사이트의 셀프 서비스 부족',
    'Network reliability': '네트워크 안정성',
    'Limited range of services': '제한된 서비스 범위',
    'Lack of affordable download/upload speed': '저렴한 다운로드/업로드 속도 부족',
    'Long distance charges': '장거리 요금',
    'Extra data charges': '추가 데이터 요금',
    "Don't know": '모르겠다',
    'Poor expertise of online support': '온라인 지원의 전문성 부족',
    'Poor expertise of phone support': '전화 지원의 전문성 부족',
    'Attitude of service provider': '서비스 제공업체의 태도',
    'Attitude of support person': '지원 담당자의 태도',
    'Deceased': '사망'
}

In [ ]:
total_final['한글이탈이유'] = total_final['이탈이유'].map(reason_mapping)
total_final.head(20)

In [ ]:
# 이탈유형별 이탈이유별 개수
reason_counts_by_type = total_final.groupby('한글이탈유형')['한글이탈이유'].value_counts()

# 결과 확인
print(reason_counts_by_type)

In [ ]:
total_final['한글이탈유형'].value_counts()

In [ ]:
# total_final.to_csv('data/total_real_final.csv', index=False, encoding='utf-8-sig')

In [25]:
# 한글 컬럼 기준으로 필터링
filtered_df = total_final[
    (total_final['한글이탈유형'] == '가격') & 
    (total_final['한글이탈이유'] == '웹사이트의 셀프 서비스 부족')
]

# 결과 확인
print(filtered_df)


            고객ID         위도          경도     성별  고령자여부  배우자여부  부양가족여부  가입개월수  \
999   1449-XQEMT  40.559177 -120.612113   True  False  False   False      3   
1346  2002-MZHWP  37.422961 -120.765493  False  False  False   False      1   
1602  2332-EFBJY  34.147149 -118.463365   True  False  False   False     16   
2977  4255-DDUOU  34.866032 -120.536546  False  False   True    True      1   
5574  7853-GVUDZ  34.076259 -118.310715  False  False   True    True      1   

      전화서비스가입여부  복수회선여부  ... 음악스트리밍이용여부  무제한데이터이용여부  총환불액  총초과데이터요금  총장거리통화요금  \
999        True   False  ...      False       False   0.0         0     45.36   
1346       True   False  ...      False       False   0.0         0     26.12   
1602       True   False  ...      False       False   0.0         0     91.84   
2977       True   False  ...      False       False   0.0         0     34.57   
5574       True   False  ...      False       False   0.0         0      3.09   

        총납부금  이탈유형_x 한글이탈유형           